In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer



# df = pd.read_csv('../films_db2.csv')
# df.head(10)

In [2]:
from db.database_mysql import engine

df = pd.read_sql_query('''SELECT jp.title, jp.date, jp.`year`, jp.director, jp.country, jp.duration, jp.genre, jp.first_day, jp.first_week, jp.first_weekend, jp.hebdo_rank, jp.total_spectator, jp.copies, fa1.distributor,fa1.rating_press, fa1.rating_public, fa1.casting, fa1.budget, fa1.lang, fa1.visa, fa1.award 
FROM films_jp_2 as jp
LEFT JOIN films_allo fa1 ON fa1.id_jp = jp.id                     
  where fa1.year_allo is not null and fa1.year_allo != -1
  group by fa1.id_jp, jp.`year`, jp.director''', engine)

df.head()

,title,date,year,director,country,duration,genre,first_day,first_week,first_weekend,...,total_spectator,copies,distributor,rating_press,rating_public,casting,budget,lang,visa,award
0,rien ne va plus,1997-10-15,1997,claude chabrol,france,6300,thriller,-1,172230,-1,...,458125,234,Carlotta Films,-1.0,3.0,"[""Michel Serrault"", ""Isabelle Huppert"", ""Franç...",-1,"[""francais"", ""hongrois""]",90016,0
1,la fille sur le pont,1999-03-31,1999,patrice leconte,france,5400,drame,-1,154881,-1,...,663390,198,,3.4,3.6,"[""Vanessa Paradis"", ""Daniel Auteuil"", ""Claude ...",-1,"[""francais""]",90876,1
2,grosse fatigue,1994-05-18,1994,michel blanc,france,5220,comedie,-1,417021,-1,...,2015230,189,,-1.0,2.7,"[""Michel Blanc"", ""Philippe Noiret"", ""Marie-Ann...",-1,"[""francais""]",82266,1
3,braveheart,1995-10-04,1995,mel gibson,etatsunis,9900,aventure action,-1,355642,-1,...,1231534,302,ufd,2.6,4.2,"[""Mel Gibson"", ""Sophie Marceau"", ""Catherine Mc...",-1,"[""anglais""]",88091,9
4,soleil trompeur,1994-08-31,1994,None,russie,9120,comedie dramatique,-1,101953,-1,...,529790,129,,-1.0,3.8,"[""Nikita Mikhalkov"", ""Oleg Menshikov"", ""Ingebo...",-1,"[""francais"", ""russe""]",-1,0


In [3]:
df.nunique()

title              3630
date               1482
year                 34
director            351
country              33
duration            119
genre                18
first_day          3031
first_week         3611
first_weekend      2856
hebdo_rank           15
total_spectator    3624
copies              757
distributor         155
rating_press         41
rating_public        40
casting            3290
budget              212
lang                223
visa               2964
award                27
dtype: int64

In [4]:
# from db.database_mysql import engine

# df = pd.read_sql_query('''SELECT jp.title, jp.`year`, jp.director, jp.country, jp.duration, jp.genre, jp.first_day, jp.first_week, jp.first_weekend, jp.hebdo_rank, jp.total_spectator, jp.copies, fa1.rating_press, fa1.rating_public, fa1.casting, fa1.budget, fa1.lang, fa1.visa, fa1.award 
# FROM films_jp as jp
# LEFT JOIN films_allo fa1 ON fa1.id_jp = jp.id 
#                         and fa1.year_allo = jp.`year` ''', engine)
# # and fa1.director_allo like CONCAT("%", jp.director, "%")

# df.head()

In [5]:
# Réalisateurs très connus
realisateurs_tres_connus = [
    "steven spielberg",
    "martin scorsese",
    "alfred hitchcock",
    "quentin tarantino",
    "stanley kubrick",
    "francis ford coppola",
    "steven soderbergh",
    "tim burton",
    "christopher nolan",
    "james cameron",
    "woody allen",
    "clint eastwood",
    "ridley scott",
    "pedro almodóvar",
    "david fincher",
    "peter jackson",
    "darren aronofsky",
    "coen brothers",
    "spike lee",
    "wes anderson",
    "roman polanski",
    "david lynch",
    "terrence malick",
    "michael bay",
    "ang lee",
    "paul thomas anderson",
    "quentin dupieux",
    "alejandro gonzález iñárritu",
    "werner herzog",
    "david cronenberg",
    "guy ritchie",
    "robert rodriguez",
    "john carpenter",
    "baz luhrmann",
    "jean-pierre jeunet",
    "gaspar noé",
    "yorgos lanthimos",
    "george lucas",
    "james wan",
    "luc besson",
    "catherine hardwicke",
    "zhang yimou",
    "john woo",
    "sofia coppola",
    "gus van sant",
    "danny boyle",
    "alejandro amenábar",
    "fernando meirelles",
    "tom tykwer"
]

# Réalisateurs moins connus
realisateurs_moins_connus = [
    "kelly reichardt",
    "todd solondz",
    "sarah polley",
    "lynne ramsay",
    "joe swanberg",
    "andrew bujalski",
    "sion sono",
    "ti west",
    "adam wingard",
    "jeremy saulnier",
    "sean baker",
    "mike flanagan",
    "chad stahelski",
    "justin benson",
    "aaron moorhead",
    "ana lily amirpour",
    "jeff nichols",
    "jon watts",
    "yorgos zois",
    "hirokazu kore-eda",
    "josh safdie",
    "benny safdie",
    "robert eggers",
    "panos cosmatos",
    "ari aster",
    "barry jenkins",
    "greta gerwig",
    "jordan peele",
    "bo burnham",
    "alex garland",
    "ryan coogler",
    "travis knight",
    "céline sciamma",
    "matthias schoenaerts",
    "laszlo nemes",
    "makoto shinkai",
    "ladj ly",
    "chloé zhao",
    "emerald fennell",
    "alice rohrwacher",
    "sam levinson",
    "brandon cronenberg",
    "julia ducournau",
    "karyn kusama",
    "justin simien",
    "lorene scafaria",
    "ava duvernay",
    "dee rees"
]

# Réalisateurs pas connus
realisateurs_pas_connus = [
    "maxime caperan",
    "sophie blondy",
    "daniel cattier",
    "julien rambaldi",
    "fabrice gobert",
    "axelle ropert",
    "léonor serraille",
    "axelle laffont",
    "yann le quellec",
    "elsa amiel",
    "matthieu bareyre",
    "nina meurisse",
    "emmanuel mouret",
    "aurélie saada",
    "arthur delaIre",
    "quentin lecocq",
    "terry gilliam",
    "yvan attal",
    "maïwenn",
    "roschdy zem",
    "michaël youn",
    "édouard deluc",
    "noémie lvovsky",
    "julia roy",
    "marina foïs",
    "éric lartigau",
    "cyril dion",
    "jacques audiard",
    "gilles lellouche",
    "patrick cassir",
    "alain chabat",
    "zabou breitman",
    "yann gozlan",
    "riad sattouf",
    "mathias malzieu",
    "guillaume nicloux",
    "eran riklis",
    "fabien onteniente",
    "safy nebbou",
    "mathieu sapin",
    "denis imbert",
    "hélène fillières",
    "lisa azuelos",
    "léa fazer",
    "albert dupontel",
    "jean dujardin",
    "audrey dana",
    "florence foresti",
    "pierre salvadori"
]

In [6]:
def classify_director(director):
    if director in realisateurs_tres_connus:
        return 3
    elif director in realisateurs_moins_connus:
        return 2
    else:
        return 1

df['director'] = df['director'].apply(classify_director)

df

,title,date,year,director,country,duration,genre,first_day,first_week,first_weekend,...,total_spectator,copies,distributor,rating_press,rating_public,casting,budget,lang,visa,award
0,rien ne va plus,1997-10-15,1997,1,france,6300,thriller,-1,172230,-1,...,458125,234,Carlotta Films,-1.0,3.0,"[""Michel Serrault"", ""Isabelle Huppert"", ""Franç...",-1,"[""francais"", ""hongrois""]",90016,0
1,la fille sur le pont,1999-03-31,1999,1,france,5400,drame,-1,154881,-1,...,663390,198,,3.4,3.6,"[""Vanessa Paradis"", ""Daniel Auteuil"", ""Claude ...",-1,"[""francais""]",90876,1
2,grosse fatigue,1994-05-18,1994,1,france,5220,comedie,-1,417021,-1,...,2015230,189,,-1.0,2.7,"[""Michel Blanc"", ""Philippe Noiret"", ""Marie-Ann...",-1,"[""francais""]",82266,1
3,braveheart,1995-10-04,1995,1,etatsunis,9900,aventure action,-1,355642,-1,...,1231534,302,ufd,2.6,4.2,"[""Mel Gibson"", ""Sophie Marceau"", ""Catherine Mc...",-1,"[""anglais""]",88091,9
4,soleil trompeur,1994-08-31,1994,1,russie,9120,comedie dramatique,-1,101953,-1,...,529790,129,,-1.0,3.8,"[""Nikita Mikhalkov"", ""Oleg Menshikov"", ""Ingebo...",-1,"[""francais"", ""russe""]",-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3627,george de la jungle,1997-10-15,1997,1,etatsunis,5520,film familial,-1,129374,-1,...,587914,248,,-1.0,2.1,"[""Brendan Fraser"", ""Leslie Mann"", ""Richard Rou...",-1,"[""anglais"", ""espagnol""]",93126,0
3628,terrain mine,1994-04-06,1994,1,etatsunis,6060,aventure action,-1,186581,-1,...,382000,264,warner bros france,-1.0,2.1,"[""Fran Monegan"", ""Gabriel L. Muktoyuk"", ""Helen...",-1,"[""anglais""]",85264,0
3629,rasta rockett,1994-04-13,1993,1,etatsunis,6480,film familial,-1,227393,-1,...,2523167,123,gaumont buena vista international gbvi,3.0,3.8,"[""Leon Robinson"", ""Doug E. Doug"", ""John Candy""...",-1,"[""anglais"", ""allemand"", ""russe""]",84550,0
3630,be happy,2008-08-27,2008,1,grandebretagne,7080,comedie,18126,110442,90372,...,362884,128,mk2 diffusion,3.9,2.7,"[""Sally Hawkins"", ""Alexis Zegerman"", ""Andrea R...",-1,"[""anglais""]",121057,1


In [7]:
# import json

# test = df['casting'].unique()[0]

# json.loads(test)

# df["casting"] = df["casting"].apply(json.loads)

In [8]:
acteurs_tres_connus = [
    "Tom Hanks", "Meryl Streep", "Leonardo DiCaprio", "Jennifer Lawrence", "Denzel Washington",
    "Cate Blanchett", "Brad Pitt", "Angelina Jolie", "Johnny Depp", "Julia Roberts",
    "Robert Downey Jr.", "Scarlett Johansson", "Will Smith", "Natalie Portman", "George Clooney",
    "Charlize Theron", "Matt Damon", "Emma Stone", "Nicole Kidman", "Christian Bale",
    "Tom Cruise", "Helen Mirren", "Daniel Day-Lewis", "Sandra Bullock", "Jake Gyllenhaal",
    "Kate Winslet", "Morgan Freeman", "Anne Hathaway", "Sean Penn", "Kate Hudson",
    "Liam Neeson", "Viola Davis", "Mark Wahlberg", "Halle Berry", "Hugh Jackman",
    "Jessica Chastain", "Chris Hemsworth", "Judi Dench", "Ryan Gosling", "Emily Blunt",
    "Eddie Redmayne", "Marion Cotillard", "James Franco", "Amy Adams", "Colin Firth",
    "Reese Witherspoon", "Jude Law", "Gwyneth Paltrow", "Ryan Reynolds", "Julianne Moore",
    "Christoph Waltz", "Rachel McAdams", "Michael Fassbender", "Anne Bancroft", "Javier Bardem",
    "Robin Williams", "Penélope Cruz", "Joaquin Phoenix", "Michelle Williams", "Mark Ruffalo",
    "Naomi Watts", "Steve Carell", "Carey Mulligan", "Kevin Spacey", "Emma Watson",
    "Woody Harrelson", "Diane Keaton", "Anthony Hopkins", "Kristen Stewart", "Jamie Foxx",
    "Alicia Vikander", "Jeremy Renner", "Maggie Smith", "Seth Rogen", "Jodie Foster",
    "Daniel Radcliffe", "Saoirse Ronan", "Ralph Fiennes", "Michelle Pfeiffer", "Jeff Bridges",
    "Salma Hayek", "Antonio Banderas", "Uma Thurman", "Donald Sutherland", "Tom Hardy",
    "Kirsten Dunst", "John Travolta", "Anne Hathaway", "Gary Oldman", "Sigourney Weaver",
    "Joaquin Phoenix", "Catherine Zeta-Jones", "Emma Thompson", "Paul Rudd", "Diane Lane",
    "John Malkovich", "Eva Green", "Colin Farrell", "Angelina Jolie", "Bradley Cooper"
]

acteurs_moins_connus = [
    "Bill Skarsgård", "Florence Pugh", "Timothée Chalamet", "Elle Fanning", "Robert Pattinson",
    "Amanda Seyfried", "Dev Patel", "Eiza González", "John David Washington", "Lily James",
    "Domhnall Gleeson", "Gemma Chan", "Nicholas Hoult", "Ana de Armas", "Timothee Chalamet",
    "Saoirse Ronan", "Anya Taylor-Joy", "Shia LaBeouf", "Mackenzie Davis", "Jessie Buckley",
    "Callum Turner", "Thomasin McKenzie", "Kelvin Harrison Jr.", "Henry Golding", "Kaitlyn Dever",
    "Billy Magnussen", "Amandla Stenberg", "Tessa Thompson", "Beanie Feldstein", "Lakeith Stanfield",
    "Olivia Cooke", "Billie Lourd", "Noah Centineo", "Yara Shahidi", "Margaret Qualley", "Riz Ahmed",
    "Letitia Wright", "Jack Lowden", "Kelvin Harrison Jr.", "Naomi Scott", "Barry Keoghan",
    "Taylor Russell", "Timothée Chalamet", "Elsie Fisher", "Joey King", "Awkwafina", "Olivia Cooke",
    "Caleb Landry Jones", "Thomasin McKenzie", "Winston Duke", "Aja Naomi King", "Noah Jupe",
    "Sophia Lillis", "Lucas Hedges", "Zendaya", "Billie Eilish", "Harris Dickinson", "Katherine Langford",
    "Nick Robinson", "Millie Bobby Brown", "Beanie Feldstein", "Noah Centineo", "Finn Wolfhard",
    "Zoey Deutch", "Anthony Ramos", "Sydney Sweeney", "Keeley Hawes", "Ross Lynch", "Euphoria",
    "Sarah Gadon", "Noah Schnapp", "Jacob Elordi", "Liana Liberato", "Shameik Moore", "Elle Fanning",
    "Kaitlyn Dever", "Sadie Sink", "Lia Marie Johnson", "Fionn Whitehead", "Katherine Waterston",
    "Nicholas Hoult", "Odeya Rush", "Kiernan Shipka", "Lucas Till", "Halston Sage", "Tom Holland",
    "Maude Apatow", "Thomas Mann", "Julia Garner", "Charlie Plummer", "Bella Thorne", "Cameron Monaghan",
    "Elle Fanning", "Jack Dylan Grazer", "Beulah Koale", "Harris Dickinson", "Katherine Langford",
    "Thomasin McKenzie", "Eddie Redmayne", "Gaten Matarazzo"
]

acteurs_fr = [
    "Jean Dujardin",
    "Marion Cotillard",
    "Vincent Cassel",
    "Audrey Tautou",
    "Omar Sy",
    "Juliette Binoche",
    "Guillaume Canet",
    "Catherine Deneuve",
    "Gérard Depardieu",
    "Isabelle Huppert",
    "Mélanie Laurent",
    "Louis de Funès",
    "Emmanuelle Béart",
    "François Cluzet",
    "Léa Seydoux",
    "Jean Reno",
    "Sophie Marceau",
    "Mathieu Amalric",
    "Vincent Lindon",
    "Bérénice Bejo",
    "Kad Merad",
    "Monica Bellucci",
    "Romain Duris",
    "Sandrine Kiberlain",
    "André Dussollier",
    "Audrey Fleurot",
    "Dany Boon",
    "Kristin Scott Thomas",
    "Fabrice Luchini",
    "Charlotte Gainsbourg",
    "Jean-Paul Belmondo",
    "Mélanie Doutey",
    "François Berléand",
    "Emmanuelle Devos",
    "Jean-Pierre Bacri",
    "Emmanuelle Seigner",
    "Lambert Wilson",
    "Valérie Lemercier",
    "Karin Viard",
    "Michel Blanc",
    "Isabelle Carré",
    "Vincent Perez",
    "Ludivine Sagnier",
    "Jean-Hugues Anglade",
    "Marina Foïs",
    "Gilles Lellouche",
    "Sara Forestier",
    "Daniel Auteuil",
    "Chiara Mastroianni",
    "Josiane Balasko"
]


In [9]:
# def classifier_acteurs(acteurs):
#     # print(type(acteurs))
#     result = 0
#     for acteur in acteurs:
#         # print(acteur,acteurs)
#         if acteur in acteurs_tres_connus:
#             result += 10
#         elif acteur in acteurs_fr:
#             result += 7
#         elif acteur in acteurs_moins_connus:
#             result += 5
#         else :
#             result += 0
#     return result
def classifier_acteurs(acteurs):
    # print(type(acteurs))

    for acteur in acteurs:
        # print(acteur,acteurs)
        if acteur in acteurs_tres_connus:
            return 3
        elif acteur in acteurs_fr:
            return 2
        elif acteur in acteurs_moins_connus:
            return 1
        else :
            return 0

df['classification_acteurs'] = df['casting'].apply(lambda x: classifier_acteurs(eval(x)))

df


,title,date,year,director,country,duration,genre,first_day,first_week,first_weekend,...,copies,distributor,rating_press,rating_public,casting,budget,lang,visa,award,classification_acteurs
0,rien ne va plus,1997-10-15,1997,1,france,6300,thriller,-1,172230,-1,...,234,Carlotta Films,-1.0,3.0,"[""Michel Serrault"", ""Isabelle Huppert"", ""Franç...",-1,"[""francais"", ""hongrois""]",90016,0,0.0
1,la fille sur le pont,1999-03-31,1999,1,france,5400,drame,-1,154881,-1,...,198,,3.4,3.6,"[""Vanessa Paradis"", ""Daniel Auteuil"", ""Claude ...",-1,"[""francais""]",90876,1,0.0
2,grosse fatigue,1994-05-18,1994,1,france,5220,comedie,-1,417021,-1,...,189,,-1.0,2.7,"[""Michel Blanc"", ""Philippe Noiret"", ""Marie-Ann...",-1,"[""francais""]",82266,1,2.0
3,braveheart,1995-10-04,1995,1,etatsunis,9900,aventure action,-1,355642,-1,...,302,ufd,2.6,4.2,"[""Mel Gibson"", ""Sophie Marceau"", ""Catherine Mc...",-1,"[""anglais""]",88091,9,0.0
4,soleil trompeur,1994-08-31,1994,1,russie,9120,comedie dramatique,-1,101953,-1,...,129,,-1.0,3.8,"[""Nikita Mikhalkov"", ""Oleg Menshikov"", ""Ingebo...",-1,"[""francais"", ""russe""]",-1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3627,george de la jungle,1997-10-15,1997,1,etatsunis,5520,film familial,-1,129374,-1,...,248,,-1.0,2.1,"[""Brendan Fraser"", ""Leslie Mann"", ""Richard Rou...",-1,"[""anglais"", ""espagnol""]",93126,0,0.0
3628,terrain mine,1994-04-06,1994,1,etatsunis,6060,aventure action,-1,186581,-1,...,264,warner bros france,-1.0,2.1,"[""Fran Monegan"", ""Gabriel L. Muktoyuk"", ""Helen...",-1,"[""anglais""]",85264,0,0.0
3629,rasta rockett,1994-04-13,1993,1,etatsunis,6480,film familial,-1,227393,-1,...,123,gaumont buena vista international gbvi,3.0,3.8,"[""Leon Robinson"", ""Doug E. Doug"", ""John Candy""...",-1,"[""anglais"", ""allemand"", ""russe""]",84550,0,0.0
3630,be happy,2008-08-27,2008,1,grandebretagne,7080,comedie,18126,110442,90372,...,128,mk2 diffusion,3.9,2.7,"[""Sally Hawkins"", ""Alexis Zegerman"", ""Andrea R...",-1,"[""anglais""]",121057,1,0.0


In [10]:
def classifier_pays(pays):
    if pays == 'etatsunis':
        return 3
    elif pays == 'france':
        return 2
    else :
        return 0

df['classification_country'] = df['country'].apply(lambda x: classifier_pays(x))

df

france
france
france
etatsunis
russie
etatsunis
grandebretagne
france
france
etatsunis
france
etatsunis
france
etatsunis
etatsunis
etatsunis
etatsunis
grandebretagne
etatsunis
grandebretagne
etatsunis
etatsunis
france
etatsunis
etatsunis
etatsunis
etatsunis
etatsunis
france
etatsunis
etatsunis
france
etatsunis
etatsunis
etatsunis
france
france
etatsunis
etatsunis
etatsunis
etatsunis
etatsunis
etatsunis
etatsunis
etatsunis
chine
france
france
etatsunis
france
etatsunis
etatsunis
etatsunis
afrique du sud
france
etatsunis
france
france
etatsunis
france
france
france
grandebretagne
france
etatsunis
france
etatsunis
etatsunis
etatsunis
grandebretagne
france
france
etatsunis
etatsunis
etatsunis
france
france
france
france
france
grandebretagne
france
singapour
france
france
etatsunis
france
france
france
etatsunis
france
etatsunis
france
etatsunis
france
etatsunis
etatsunis
etatsunis
allemagne
france
etatsunis
etatsunis
france
france
france
etatsunis
etatsunis
etatsunis
etatsunis
etatsunis
j

,title,date,year,director,country,duration,genre,first_day,first_week,first_weekend,...,distributor,rating_press,rating_public,casting,budget,lang,visa,award,classification_acteurs,classification_country
0,rien ne va plus,1997-10-15,1997,1,france,6300,thriller,-1,172230,-1,...,Carlotta Films,-1.0,3.0,"[""Michel Serrault"", ""Isabelle Huppert"", ""Franç...",-1,"[""francais"", ""hongrois""]",90016,0,0.0,2
1,la fille sur le pont,1999-03-31,1999,1,france,5400,drame,-1,154881,-1,...,,3.4,3.6,"[""Vanessa Paradis"", ""Daniel Auteuil"", ""Claude ...",-1,"[""francais""]",90876,1,0.0,2
2,grosse fatigue,1994-05-18,1994,1,france,5220,comedie,-1,417021,-1,...,,-1.0,2.7,"[""Michel Blanc"", ""Philippe Noiret"", ""Marie-Ann...",-1,"[""francais""]",82266,1,2.0,2
3,braveheart,1995-10-04,1995,1,etatsunis,9900,aventure action,-1,355642,-1,...,ufd,2.6,4.2,"[""Mel Gibson"", ""Sophie Marceau"", ""Catherine Mc...",-1,"[""anglais""]",88091,9,0.0,3
4,soleil trompeur,1994-08-31,1994,1,russie,9120,comedie dramatique,-1,101953,-1,...,,-1.0,3.8,"[""Nikita Mikhalkov"", ""Oleg Menshikov"", ""Ingebo...",-1,"[""francais"", ""russe""]",-1,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3627,george de la jungle,1997-10-15,1997,1,etatsunis,5520,film familial,-1,129374,-1,...,,-1.0,2.1,"[""Brendan Fraser"", ""Leslie Mann"", ""Richard Rou...",-1,"[""anglais"", ""espagnol""]",93126,0,0.0,3
3628,terrain mine,1994-04-06,1994,1,etatsunis,6060,aventure action,-1,186581,-1,...,warner bros france,-1.0,2.1,"[""Fran Monegan"", ""Gabriel L. Muktoyuk"", ""Helen...",-1,"[""anglais""]",85264,0,0.0,3
3629,rasta rockett,1994-04-13,1993,1,etatsunis,6480,film familial,-1,227393,-1,...,gaumont buena vista international gbvi,3.0,3.8,"[""Leon Robinson"", ""Doug E. Doug"", ""John Candy""...",-1,"[""anglais"", ""allemand"", ""russe""]",84550,0,0.0,3
3630,be happy,2008-08-27,2008,1,grandebretagne,7080,comedie,18126,110442,90372,...,mk2 diffusion,3.9,2.7,"[""Sally Hawkins"", ""Alexis Zegerman"", ""Andrea R...",-1,"[""anglais""]",121057,1,0.0,0


In [11]:
# Studios très connus
distributeurs_tres_connus = [
    'United International Pictures (UIP)',
    'Gaumont Buena Vista International (GBVI)',
    'Sony Pictures Releasing France',
    'Warner Bros. France',
    'The Walt Disney Company France',
    'Twentieth Century Fox France',
    'Columbia TriStar Films',
    '20th Century Studios',
    'Universal Pictures International France',
    'Pathé',
    'StudioCanal',
    'EuropaCorp Distribution',
    'Paramount Pictures France',
    'Universal Pictures France (UPF)'
]

# Studios moins connus
distributeurs_moins_connus = [
    'MK2 Diffusion',
    'Gaumont Distribution',
    'PolyGram Film Distribution',
    'Sony Pictures Home Entertainment',
    'Carlotta Films',
    'Bac Films',
    'Tamasa Distribution',
    'Mars Distribution',
    'Diaphana Films',
    'Wild Bunch Distribution',
    'Metropolitan FilmExport',
    'ARP Sélection',
    'SND',
    'Haut et Court',
    'Memento Distribution',
    'Le Pacte',
    'Ad Vitam',
    'Happiness Distribution',
    'Les Films du Losange',
    'Rezo Films',
    'Gebeka Films',
    'Les Films Number One',
    'Mary-X Distribution',
    'ARP / UGC',
    'Colifilms Diffusion',
    'Splendor Films',
    'AMLF',
    'UFD'
]




In [12]:
def classifier_distributeurs(distributeur):
    if distributeur in distributeurs_tres_connus:
        return 1
    elif distributeur in distributeurs_moins_connus:
        return 2
    else:
        return 3

df['distributor'] = df['distributor'].apply(classifier_distributeurs)

df

,title,date,year,director,country,duration,genre,first_day,first_week,first_weekend,...,distributor,rating_press,rating_public,casting,budget,lang,visa,award,classification_acteurs,classification_country
0,rien ne va plus,1997-10-15,1997,1,france,6300,thriller,-1,172230,-1,...,2,-1.0,3.0,"[""Michel Serrault"", ""Isabelle Huppert"", ""Franç...",-1,"[""francais"", ""hongrois""]",90016,0,0.0,2
1,la fille sur le pont,1999-03-31,1999,1,france,5400,drame,-1,154881,-1,...,3,3.4,3.6,"[""Vanessa Paradis"", ""Daniel Auteuil"", ""Claude ...",-1,"[""francais""]",90876,1,0.0,2
2,grosse fatigue,1994-05-18,1994,1,france,5220,comedie,-1,417021,-1,...,3,-1.0,2.7,"[""Michel Blanc"", ""Philippe Noiret"", ""Marie-Ann...",-1,"[""francais""]",82266,1,2.0,2
3,braveheart,1995-10-04,1995,1,etatsunis,9900,aventure action,-1,355642,-1,...,3,2.6,4.2,"[""Mel Gibson"", ""Sophie Marceau"", ""Catherine Mc...",-1,"[""anglais""]",88091,9,0.0,3
4,soleil trompeur,1994-08-31,1994,1,russie,9120,comedie dramatique,-1,101953,-1,...,3,-1.0,3.8,"[""Nikita Mikhalkov"", ""Oleg Menshikov"", ""Ingebo...",-1,"[""francais"", ""russe""]",-1,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3627,george de la jungle,1997-10-15,1997,1,etatsunis,5520,film familial,-1,129374,-1,...,3,-1.0,2.1,"[""Brendan Fraser"", ""Leslie Mann"", ""Richard Rou...",-1,"[""anglais"", ""espagnol""]",93126,0,0.0,3
3628,terrain mine,1994-04-06,1994,1,etatsunis,6060,aventure action,-1,186581,-1,...,3,-1.0,2.1,"[""Fran Monegan"", ""Gabriel L. Muktoyuk"", ""Helen...",-1,"[""anglais""]",85264,0,0.0,3
3629,rasta rockett,1994-04-13,1993,1,etatsunis,6480,film familial,-1,227393,-1,...,3,3.0,3.8,"[""Leon Robinson"", ""Doug E. Doug"", ""John Candy""...",-1,"[""anglais"", ""allemand"", ""russe""]",84550,0,0.0,3
3630,be happy,2008-08-27,2008,1,grandebretagne,7080,comedie,18126,110442,90372,...,3,3.9,2.7,"[""Sally Hawkins"", ""Alexis Zegerman"", ""Andrea R...",-1,"[""anglais""]",121057,1,0.0,0


In [13]:
df["classification_acteurs"].nunique()

4

In [14]:
df.isnull().sum()

title                       0
date                        0
year                        0
director                    0
country                     0
duration                    0
genre                       0
first_day                   0
first_week                  0
first_weekend               0
hebdo_rank                  0
total_spectator             0
copies                      0
distributor                 0
rating_press                0
rating_public               0
casting                     0
budget                      0
lang                        0
visa                        0
award                       0
classification_acteurs    320
classification_country      0
dtype: int64

In [15]:
colonnes_numeriques = ["first_day", "rating_press", "rating_public", "first_weekend",'budget']

df[colonnes_numeriques] = df[colonnes_numeriques].replace(-1, np.nan)

imputer = KNNImputer(n_neighbors=5)

df_imputed = imputer.fit_transform(df[colonnes_numeriques])

df[colonnes_numeriques] = df_imputed

df.head()

,title,date,year,director,country,duration,genre,first_day,first_week,first_weekend,...,distributor,rating_press,rating_public,casting,budget,lang,visa,award,classification_acteurs,classification_country
0,rien ne va plus,1997-10-15,1997,1,france,6300,thriller,62681.6,172230,429428.6,...,2,3.28,3.0,"[""Michel Serrault"", ""Isabelle Huppert"", ""Franç...",89400000.0,"[""francais"", ""hongrois""]",90016,0,0.0,2
1,la fille sur le pont,1999-03-31,1999,1,france,5400,drame,45454.6,154881,920127.2,...,3,3.40,3.6,"[""Vanessa Paradis"", ""Daniel Auteuil"", ""Claude ...",165600000.0,"[""francais""]",90876,1,0.0,2
2,grosse fatigue,1994-05-18,1994,1,france,5220,comedie,183871.2,417021,258968.6,...,3,2.70,2.7,"[""Michel Blanc"", ""Philippe Noiret"", ""Marie-Ann...",906030000.0,"[""francais""]",82266,1,2.0,2
3,braveheart,1995-10-04,1995,1,etatsunis,9900,aventure action,172335.2,355642,840442.0,...,3,2.60,4.2,"[""Mel Gibson"", ""Sophie Marceau"", ""Catherine Mc...",208200000.0,"[""anglais""]",88091,9,0.0,3
4,soleil trompeur,1994-08-31,1994,1,russie,9120,comedie dramatique,45689.0,101953,553612.6,...,3,3.14,3.8,"[""Nikita Mikhalkov"", ""Oleg Menshikov"", ""Ingebo...",131000000.0,"[""francais"", ""russe""]",-1,0,0.0,0


In [16]:
df.isnull().sum()

title                       0
date                        0
year                        0
director                    0
country                     0
duration                    0
genre                       0
first_day                   0
first_week                  0
first_weekend               0
hebdo_rank                  0
total_spectator             0
copies                      0
distributor                 0
rating_press                0
rating_public               0
casting                     0
budget                      0
lang                        0
visa                        0
award                       0
classification_acteurs    320
classification_country      0
dtype: int64

In [17]:
df.nunique()

title                     3630
date                      1482
year                        34
director                     3
country                     33
duration                   119
genre                       18
first_day                 3308
first_week                3611
first_weekend             3244
hebdo_rank                  15
total_spectator           3624
copies                     757
distributor                  3
rating_press               130
rating_public               57
casting                   3290
budget                     944
lang                       223
visa                      2964
award                       27
classification_acteurs       4
classification_country       3
dtype: int64

In [18]:
df['date'] = pd.to_datetime(df['date'])

df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

df.drop("date", axis=1, inplace=True)

In [19]:
df.to_csv('Dataset_analyse.csv', index=False)
df

,title,year,director,country,duration,genre,first_day,first_week,first_weekend,hebdo_rank,...,rating_public,casting,budget,lang,visa,award,classification_acteurs,classification_country,day,month
0,rien ne va plus,1997,1,france,6300,thriller,62681.6,172230,429428.6,3,...,3.0,"[""Michel Serrault"", ""Isabelle Huppert"", ""Franç...",89400000.0,"[""francais"", ""hongrois""]",90016,0,0.0,2,15,10
1,la fille sur le pont,1999,1,france,5400,drame,45454.6,154881,920127.2,5,...,3.6,"[""Vanessa Paradis"", ""Daniel Auteuil"", ""Claude ...",165600000.0,"[""francais""]",90876,1,0.0,2,31,3
2,grosse fatigue,1994,1,france,5220,comedie,183871.2,417021,258968.6,2,...,2.7,"[""Michel Blanc"", ""Philippe Noiret"", ""Marie-Ann...",906030000.0,"[""francais""]",82266,1,2.0,2,18,5
3,braveheart,1995,1,etatsunis,9900,aventure action,172335.2,355642,840442.0,3,...,4.2,"[""Mel Gibson"", ""Sophie Marceau"", ""Catherine Mc...",208200000.0,"[""anglais""]",88091,9,0.0,3,4,10
4,soleil trompeur,1994,1,russie,9120,comedie dramatique,45689.0,101953,553612.6,7,...,3.8,"[""Nikita Mikhalkov"", ""Oleg Menshikov"", ""Ingebo...",131000000.0,"[""francais"", ""russe""]",-1,0,0.0,0,31,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3627,george de la jungle,1997,1,etatsunis,5520,film familial,106952.8,129374,233040.2,4,...,2.1,"[""Brendan Fraser"", ""Leslie Mann"", ""Richard Rou...",100200000.0,"[""anglais"", ""espagnol""]",93126,0,0.0,3,15,10
3628,terrain mine,1994,1,etatsunis,6060,aventure action,106952.8,186581,233040.2,4,...,2.1,"[""Fran Monegan"", ""Gabriel L. Muktoyuk"", ""Helen...",100200000.0,"[""anglais""]",85264,0,0.0,3,6,4
3629,rasta rockett,1994,1,etatsunis,6480,film familial,53816.6,227393,695918.8,3,...,3.8,"[""Leon Robinson"", ""Doug E. Doug"", ""John Candy""...",492240000.0,"[""anglais"", ""allemand"", ""russe""]",84550,0,0.0,3,13,4
3630,be happy,2008,1,grandebretagne,7080,comedie,18126.0,110442,90372.0,9,...,2.7,"[""Sally Hawkins"", ""Alexis Zegerman"", ""Andrea R...",283600000.0,"[""anglais""]",121057,1,0.0,0,27,8
